# [Exploration 6] 멋진 작사가 만들기

## Step 1. 데이터 다운로드

먼저 데이터셋을 아래 명령어를 통해 다운로드하고 디렉토리를 설정한다.

`$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip`   
`$ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics  #lyrics 폴더에 압축풀기`

## Step 2. 데이터 읽어오기

저장한 파일을 한 문장씩 끊어서 불러온다.   
(풀잎스쿨 파이썬 시간에 배운 `open.read()`함수가 나와서 반가웠다.)

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I. LIFE.', '', '']


총 데이터는 **187,088**개의 문장을 가지고 있다. 

## Step 3. 데이터 정제

기존 문장을 토큰화 하는 과정에서 불필요한 특수문자 혹은 문장부호와 단어가 합쳐져 같은 단어임에도 다른 단어로 인식하는 경우가 있다.   
이를 방지하기 위하여 데이터를 전처리 해주는 과정이다.

In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os


def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([.])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z.]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


특수문자 양쪽에 빈 공간을 주어 단어와 특수문자가 한 단어로 인식되는 것을 방지하고,   
문장의 시작과 끝을 구별하는 `<start>`와 `<end>`도 추가하였다. (각각 **소스 문장, 타겟 문장**)

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if sentence == 'u': continue
        
    corpus.append(preprocess_sentence(sentence))
        
len(corpus)

175749

추가적으로 문장 길이가 0이거나, :(콜론)으로 끝나는 경우의 문장도 제거하여 입력, 출력을 포함하여 총 175749의 문장이 준비되었다.

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=len(corpus),  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
len(tensor)

[[    2     4    19 ...     0     0     0]
 [    2     4    19 ...     0     0     0]
 [    2  2754    19 ...     0     0     0]
 ...
 [    2   204 13534 ...     0     0     0]
 [    2    73     6 ...     0     0     0]
 [    2    13     5 ...     0     0     0]] <keras_preprocessing.text.Tokenizer object at 0x7fc357693d90>


175749

tensorflow의 `tf.keras.preprocessing.text.Tokenizer`패키지는 전처리된 데이터의 토큰화, dictionary화, 벡터화까지 한번에 할수 있는 아주 유용한 기능이다.   
구글링하여 찾아보니 **특수문자 처리 기능, 소문자 기능, 토큰화 기능** 등이 포함되어 있었다. 하지만 이번 노드에서는 활용하지 않고 넘어가겠다.   
   
   
   
*즉, `tf.keras.preprocessing.text.Tokenizer`패키지는 텐서플로우의 `'Tokenizer()`에서 확장된 패키지로 보임!*   
   
   
`fits_on_text()`는 문자 데이터를 입력받아 리스트 형태로 반환하고,   
`texts_to_sequences()`는 토큰화된 문장을 시퀀스 형태로 바꿔준다. (참고 링크 : https://codetorial.net/tensorflow/natural_language_processing_in_tensorflow_01.html)

In [5]:
print(tensor[:3, :10])

[[   2    4   19   99   19    3    0    0    0    0]
 [   2    4   19    3    0    0    0    0    0    0]
 [   2 2754   19    3    0    0    0    0    0    0]]


위에서 살펴봤듯이, 토큰화되고 시퀀스된 문장 3개의 형태이다. max_len을 15로 설정하였기떄문에 길이는 15가 되지만, 10길이까지만 확인해도 될만큼 짧은 문장이었던것 같다.

In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10:break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : the
6 : you
7 : and
8 : a
9 : to
10 : it


word to index형태로 되어있던 문장을 index to word형태로 바꿔준다.

In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 
#마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[ 2  4 19 99 19  3  0  0  0  0  0  0  0  0]
[ 4 19 99 19  3  0  0  0  0  0  0  0  0  0]


입력 문장과 출력 문장을 구분한다.   
입력 문장은 `<start>`로 시작하기에 마지막에 있는 `<end>`를 슬라이싱 해주고,   
출력 문장은 `<start>`를 슬라이싱 해준다.

## Step 4. 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input[:140000],
                                                          tgt_input[:140000],
                                                          train_size = 0.8)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (112000, 14)
Target Train: (112000, 14)


`scikit learn`의 `train_test_split`의 모듈을 사용하여 train set과 test set의 분리 즉,   
이 경우에는 입력 문장과 출력 문장으로 구분하였다.

In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

이 부분에서 `BUFFER_SIZE`가 하는 역할을 잘 이해하지 못했다.   
설명을 더 찾아보고 있다.

## Step 5. 인공지능 만들기

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
hidden_size = 512
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

이번 프로젝트에서는 LSTM 모델을 사용하여 학습을 시킬 예정이다. 하이퍼파라미터로는 embedding size와 hidden size가 있다.   
적절한 embedding과 hidden layer size를 줘야 최적의 결과를 내보일 수 있다.   
   
   의 인덱스가 들어 있습니다. Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 줍니다. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용됩니다.
초기값으로는 256과 1028로 embedding과 hidden size를 주었는데 epoch시 시간이 너무 오래걸려 128, 512로 값을 낮추어 진행하였다. (그래도 에폭당 4분 정도의 시간이 소요되었다)

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)


In [24]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  22496000  
_________________________________________________________________
lstm (LSTM)                  multiple                  394240    
_________________________________________________________________
lstm_1 (LSTM)                multiple                  525312    
_________________________________________________________________
dense (Dense)                multiple                  45167750  
Total params: 68,583,302
Trainable params: 68,583,302
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(dataset, epochs=10)

Epoch 1/10
875/875 [==============================] - 242s 276ms/step - loss: 2.5994
Epoch 2/10
875/875 [==============================] - 243s 277ms/step - loss: 2.5339
Epoch 3/10
875/875 [==============================] - 246s 281ms/step - loss: 2.4752
Epoch 4/10
875/875 [==============================] - 245s 280ms/step - loss: 2.4221
Epoch 5/10
875/875 [==============================] - 246s 281ms/step - loss: 2.3740
Epoch 6/10
875/875 [==============================] - 244s 279ms/step - loss: 2.3309
Epoch 7/10
875/875 [==============================] - 243s 278ms/step - loss: 2.2910
Epoch 8/10
875/875 [==============================] - 242s 277ms/step - loss: 2.2550
Epoch 9/10
875/875 [==============================] - 242s 277ms/step - loss: 2.2218
Epoch 10/10
875/875 [==============================] - 242s 276ms/step - loss: 2.1904


model summary를 통해 모델이 잘 설정된 걸 확인하고, 학습을 진행하였다.   
epoch이 너무 커지면 학습시간이 너무 길어져 10정도로 주었는데 다행히 루브릭 지표에 딱 맞는 2.19의 로스가 나왔다.   
아쉬운 점은 아직 오버피팅이 나온것 같지가 않아 학습을 더 시키면 로스가 낮아질 수도 있었는데, 더 시키지 못하는게 아쉽다.

### 모델이 생성한 가사 확인해보기

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
generate_text(model, tokenizer, init_sentence="<start> you are" ,max_len=20)

'<start> you are the one that i m gonna do <end> '

노드에서는 **i love**를 넣었을 떄의 가사를 테스트 했었는데 이때는 **i love you baby**라고 나왔었다.   
작곡이라고 하기엔 너무 형편..없어서 **you are**을 넣었는데, 말이 되는지?는 잘 모르겠지만 그래도 꽤 문장같은 문장이 나왔다.   
이것으로 6번째 프로젝트를 마무리 하겠다.

# 회고:  
## 1. **이번 프로젝트에서 어려웠던 점**   
`Tokenizer()`에 대한 개념과 `embedding`에 대한 개념을 이해하는데 조금 어려움이 있었다. 사람이 학습하는게 아닌 컴퓨터, 머신이 학습하는 과정이니 숫자만 인식할 수 있는 컴퓨터는 당연히 임베딩과 토크나이저, 인덱스화가 필요한데, 기본 지식이 없다보니 왜 그런 과정이 들어가는지에 대한 이해가 필요했다. 다행히 지금은 충분히 이해가 되었다 :)


   
## 2. **프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점**   
개인적으로는 **Step 3**에서  `Tokenize`함수에 대한 구글링을 많이 했다. 
`tf.keras.preprocessing.text.Tokenizer`패키지에 관한 기능과 역할, 그 이후에 `fits_on_text()` 모듈이 왜 쓰였으며 이전 과정과는 어떤 연관성이 있는지, 순서가 중요한건지 등을 디테일하게 공부하였고, 또 `texts_to_sequences()`에 대한 것도 다른 예문을 가져와 공부하였다. 다른 연습 쥬피터에는 이를 활용한 테스트들을 올려두어 나중에 복습자료로 사용할 수 있게 준비해두었다.



## 3. **루브릭 평가 지표를 맞추기 위해 시도한 것들**   

### 1. 가사 텍스트 생성 모델이 정상적으로 동작하는가? - 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가? ###   

위에서 보았듯이, 꽤 문장다운 문장을 구성하였다. 사실 기대하기로는 조금 더 길고 흐름이 있는 (시퀀스가 있는!) 문장을 생성할 줄 알았는데 생각보다 짧은 문장이라 아쉬움이 남는다.   

### 2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가? - 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가? ###   
   
진행하였다.

### 3. 텍스트 생성모델이 안정적으로 학습되었는가? - 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가? ###   
   
예제로 준 텍스트 경우에는 짧은 문장이 생성되었는데, 다른 예로 **you are**을 집어 넣었더니 꽤 안정적인 텍스트 문장이 생성되었다. validation loss를 낮추기 위해 하이퍼파라미터와 epoch값을 조정하여 2.19의 validaiton loss값을 얻었다.



   
### 4. **만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정**   
큰 어려움은 없었지만, GPU를 사용하여 학습함에도 불구하고 time/epoch 시간이 너무 길어져 다양한 하이퍼파라미터를 적용시키고 시도해보지 못한 것이 아쉽다.


   
### 5. **자기 다짐**   
프로젝트를 수행하고나서 이 프로젝트에 관한 내용을 100% 내것으로 만드는 사람이 많지 않을 것이라 생각한다 (나도 그렇다). 그렇지만 전체적인 흐름과 코드의 사용을 이해했다면 나중에 비슷한 프로젝트를 진행할 시에 참고자료로 활용할 수 있다는 건 큰 발전이라고 생각한다.   
이번 노드는 4번 프로젝트와 비슷한 자연어에 관한 노드이지만 문장을 생성한다는 독특한 자연어 처리였다. 새로운 코드를 만들어 구현했다기 보다는 여러 패키지를 사용해서 편하게 진행하였지만, 이런저런 패키지도 있다는 것을 확인했고 그 내용에 관해서도 충분히 이해를 하였으므로 많은 공부가 되었다. 앞으로 이런 식으로 프로젝트를 진행할 수만 있다면 (내 능력이 된다면) 더 발전할 수 있을거라 생각한다.




   